In [1]:
import numpy as np
import pandas as pd
import datetime
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)

from itertools import cycle
from collections import Counter

In [2]:
#available here: https://www.kaggle.com/rohitrox/healthcare-provider-fraud-detection-analysis

### Various dataframes:

In [3]:
test_provider_data = pd.read_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/datasets/Test.csv', low_memory=False)   
test_bene_data = pd.read_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/datasets/Test_Beneficiarydata.csv', low_memory=False)   
test_inp_data = pd.read_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/datasets/Test_Inpatientdata.csv', low_memory=False)   
test_outp_data = pd.read_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/datasets/Test_Outpatientdata.csv', low_memory=False)   

train_provider_data = pd.read_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/datasets/Train.csv', low_memory=False)   
train_bene_data = pd.read_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/datasets/Train_Beneficiarydata.csv', low_memory=False)   
train_inp_data = pd.read_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/datasets/Train_Inpatientdata.csv', low_memory=False)    
train_outp_data = pd.read_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/datasets/Train_Outpatientdata.csv', low_memory=False)    

In [4]:
print("There are %.0f observations in the provider test data." % len(test_provider_data))
test_provider_data.head()

There are 1353 observations in the provider test data.


,Provider
0,PRV51002
1,PRV51006
2,PRV51009
3,PRV51010
4,PRV51018


In [5]:
print("There are %.0f observations in the training provider data." % len(train_provider_data))
train_provider_data.head()

There are 5410 observations in the training provider data.


,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No


In [6]:
# the test set does not list potential fraud. 
# We will prepare the training set for study.
# Will we separately apply the same steps to the test data. 

In [7]:
test_bene_data.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70
1,BENE11007,1940-09-01,2009-12-01,1,2,0,45,610,12,12,1,1,2,2,2,2,1,2,1,1,2,0,0,1490,160
2,BENE11010,1936-07-01,NaN,2,1,0,41,30,12,12,2,1,2,1,1,2,1,1,1,2,2,0,0,1170,660
3,BENE11011,1914-03-01,NaN,2,2,0,1,360,12,12,2,1,1,2,2,1,1,2,2,1,1,5000,1068,250,320
4,BENE11014,1938-04-01,NaN,2,1,Y,45,780,12,12,2,1,1,2,1,1,2,1,2,2,2,21260,2136,120,100


In [8]:
train_bene_data.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,2,2,2,2,2,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,1,2,2,2,2,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,2,2,2,2,1,2,1,2,2,2,2,0,0,1790,1200


### Missing data notes:
* ##### We will transform DOD and can remove claim procedure codes 4-6 from data
* Beneficiary data:
    * missing data for deaths
* Test Outpatient data:
  * higher Claim Diagnosis/Procedure Codes means more missing data
  * no test information on Claim Procedure Code 4-6, and barely any for 1-3 
* Train Outpatient data:
  * no test information on Claim Procedure Code 5-6, and barely any for 1-4
* Test Inpatient data:
  * no test information on Claim Procedure Code 6
  * less missing data than outpatient in general 
* Test Inpatient data: 
  * no test information on Claim Procedure Code 6


In [9]:
def perc_nan(df):
    percnan=(df.isnull().sum(axis=0))/len(df)
    print(percnan)

# test
perc_nan(test_bene_data)
perc_nan(test_provider_data)
perc_nan(test_outp_data)
perc_nan(test_inp_data)

# train
perc_nan(train_bene_data)
perc_nan(train_provider_data)
perc_nan(train_outp_data)
perc_nan(train_inp_data)


BeneID                             0.000000
DOB                                0.000000
DOD                                0.991027
Gender                             0.000000
Race                               0.000000
RenalDiseaseIndicator              0.000000
State                              0.000000
County                             0.000000
NoOfMonths_PartACov                0.000000
NoOfMonths_PartBCov                0.000000
ChronicCond_Alzheimer              0.000000
ChronicCond_Heartfailure           0.000000
ChronicCond_KidneyDisease          0.000000
ChronicCond_Cancer                 0.000000
ChronicCond_ObstrPulmonary         0.000000
ChronicCond_Depression             0.000000
ChronicCond_Diabetes               0.000000
ChronicCond_IschemicHeart          0.000000
ChronicCond_Osteoporasis           0.000000
ChronicCond_rheumatoidarthritis    0.000000
ChronicCond_stroke                 0.000000
IPAnnualReimbursementAmt           0.000000
IPAnnualDeductibleAmt           

In [10]:
claims_dat = [train_outp_data, train_inp_data, test_outp_data, test_inp_data]
null_codes = ['ClmProcedureCode_6', 'ClmProcedureCode_5', 'ClmProcedureCode_4']

train_outp_data = train_outp_data.drop(null_codes, axis=1)
train_inp_data = train_inp_data.drop(null_codes, axis=1)
test_outp_data = test_outp_data.drop(null_codes, axis=1)
test_inp_data = test_inp_data.drop(null_codes, axis=1)

In [11]:
test_inp_data.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3
0,BENE11014,CLM67387,2009-09-09,2009-09-16,PRV57070,9000,PHY317786,PHY427017,NaN,2009-09-09,5789,1068.0,2009-09-16,332,5780,5533,496,V420,40390,2851,5990,570,41071,4280,4443.0,5849.0,NaN
1,BENE11017,CLM31237,2008-12-25,2009-01-08,PRV54750,14000,PHY314656,PHY426644,NaN,2008-12-25,5939,1068.0,2009-01-08,661,1889,41071,5990,5601,4588,5845,4549,29570,34831,NaN,5551.0,NaN,NaN
2,BENE11026,CLM78930,2009-12-09,2009-12-13,PRV53758,2000,PHY349495,NaN,NaN,2009-12-09,4019,1068.0,2009-12-13,241,4010,78791,60000,41401,V1254,4372,78650,7813,4254,NaN,NaN,NaN,NaN
3,BENE11031,CLM56810,2009-06-23,2009-07-06,PRV55825,16000,PHY429538,PHY371893,NaN,2009-06-23,8208,1068.0,2009-07-06,564,8208,4168,920,5990,40391,2859,4254,41400,5849,41401,8152.0,3320.0,NaN
4,BENE11085,CLM34625,2009-01-20,2009-01-31,PRV52338,19000,PHY397161,NaN,NaN,2009-01-20,4279,1068.0,2009-01-31,880,29654,V142,78702,30503,V140,V4582,V6109,7242,NaN,NaN,NaN,NaN,NaN


In [12]:
train_inp_data.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,6186,1068.0,2009-09-02,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,NaN,NaN
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,29590,1068.0,2009-09-20,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,431,1068.0,2009-02-22,067,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN,331.0,NaN,NaN
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,78321,1068.0,2009-08-30,975,042,3051,34400,5856,42732,486,5119,29620,20300,NaN,3893.0,NaN,NaN


In [13]:
test_outp_data.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11001,CLM392397,2009-06-02,2009-06-02,PRV55962,30,PHY347633,NaN,PHY347633,V5832,V5861,2724,3182,V5869,42731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,BENE11001,CLM430760,2009-06-23,2009-06-23,PRV56112,30,PHY381777,NaN,PHY381777,9594,E9174,4019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,BENE11007,CLM233081,2009-03-07,2009-03-07,PRV56979,200,PHY425311,NaN,PHY425311,7248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11007,CLM496381,2009-07-29,2009-07-29,PRV56573,10,PHY393253,PHY347995,NaN,58889,2449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5939
4,BENE11007,CLM521391,2009-08-12,2009-08-12,PRV56573,10,PHY417685,NaN,PHY382041,V666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [14]:
train_outp_data.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,V5866,V1272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,NaN,NaN,6115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,NaN,NaN,2723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,30000,72887,4280,7197,V4577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947


### Modifying original dataframes

In [15]:
#removing letters:
for df in [train_bene_data, train_inp_data, train_outp_data, test_bene_data, test_inp_data, test_outp_data]:
    df['BeneID'] = df['BeneID'].str.extract('(\d+)', expand=False)

for df in [train_inp_data, train_outp_data, train_provider_data, test_inp_data, test_outp_data, test_provider_data]:
    df['Provider'] = df['Provider'].str.extract('(\d+)', expand=False)
   
for df in [train_inp_data, train_outp_data, test_inp_data, test_outp_data]:
    df['ClaimID'] = df['ClaimID'].str.extract('(\d+)', expand=False)
    df['AttendingPhysician'] = df['AttendingPhysician'].str.extract('(\d+)', expand=False)
    df['OperatingPhysician'] = df['OperatingPhysician'].str.extract('(\d+)', expand=False)
    df['OtherPhysician'] = df['OtherPhysician'].str.extract('(\d+)', expand=False)


In [16]:
# creating inpatient / outpatient identifier:

for df in [train_inp_data, test_inp_data]:
    df['In_Out'] = 1
    
for df in [train_outp_data, test_outp_data]:
    df['In_Out'] = 0

In [17]:
# chronic conditions to binary:
chronic_cond_list = ['ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke']

for col in chronic_cond_list:
    train_bene_data[col] = train_bene_data[col].replace(2, 0)
    test_bene_data[col] = test_bene_data[col].replace(2, 0)
        

In [18]:
# to datetime
# TRAIN
train_bene_data['DOB'] = pd.to_datetime(train_bene_data['DOB'])
train_inp_data['ClaimStartDt'] = pd.to_datetime(train_inp_data['ClaimStartDt'])
train_inp_data['ClaimEndDt'] = pd.to_datetime(train_inp_data['ClaimEndDt'])
train_inp_data['AdmissionDt'] = pd.to_datetime(train_inp_data['AdmissionDt'])
train_inp_data['DischargeDt'] = pd.to_datetime(train_inp_data['DischargeDt'])

train_outp_data['ClaimStartDt'] = pd.to_datetime(train_outp_data['ClaimStartDt'])
train_outp_data['ClaimEndDt'] = pd.to_datetime(train_outp_data['ClaimEndDt'])

# TEST
test_bene_data['DOB'] = pd.to_datetime(test_bene_data['DOB'])
test_inp_data['ClaimStartDt'] = pd.to_datetime(test_inp_data['ClaimStartDt'])
test_inp_data['ClaimEndDt'] = pd.to_datetime(test_inp_data['ClaimEndDt'])
test_inp_data['AdmissionDt'] = pd.to_datetime(test_inp_data['AdmissionDt'])
test_inp_data['DischargeDt'] = pd.to_datetime(test_inp_data['DischargeDt'])

train_outp_data['ClaimStartDt'] = pd.to_datetime(train_outp_data['ClaimStartDt'])
train_outp_data['ClaimEndDt'] = pd.to_datetime(train_outp_data['ClaimEndDt'])

# Duration: new column
# TRAIN
train_inp_data['Duration'] = train_inp_data.DischargeDt - train_inp_data.AdmissionDt

durations = []
train_inp_data['Duration'] = train_inp_data['Duration'].astype('str')

for duration in train_inp_data['Duration']:
    if duration == 'NaT':
        durations.append([0])
    else:
        durations.append([int(s) for s in duration.split() if s.isdigit()])
    
durations = [val for sublist in durations for val in sublist]
train_inp_data['Duration'] = durations

# TEST
test_inp_data['Duration'] = test_inp_data.DischargeDt - test_inp_data.AdmissionDt

durations = []
test_inp_data['Duration'] = test_inp_data['Duration'].astype('str')

for duration in test_inp_data['Duration']:
    if duration == 'NaT':
        durations.append([0])
    else:
        durations.append([int(s) for s in duration.split() if s.isdigit()])
    
durations = [val for sublist in durations for val in sublist]
test_inp_data['Duration'] = durations


# Death column as binary variable
# TRAIN 
train_bene_data['DOD'] = train_bene_data['DOD'].astype('str')
train_bene_data['DOD'].fillna(0)

death = []
for i, row in train_bene_data.iterrows():
    if train_bene_data['DOD'][i] == 'nan':
        death.append(0)
    else:
        death.append(1)
train_bene_data['Death'] = death

# TEST 
test_bene_data['DOD'] = test_bene_data['DOD'].astype('str')
test_bene_data['DOD'].fillna(0)

death = []
for i, row in test_bene_data.iterrows():
    if test_bene_data['DOD'][i] == 'nan':
        death.append(0)
    else:
        death.append(1)
test_bene_data['Death'] = death


# Creating age column. 
# TRAIN
train_bene_data['DOD'] = pd.to_datetime(train_bene_data['DOD'])
train_bene_data['Age'] = round(((train_bene_data['DOD'] - train_bene_data['DOB']).dt.days)/365)
train_bene_data['Age'] = train_bene_data['Age'].fillna(round(((train_bene_data['DOD'].max() - train_bene_data['DOB']).dt.days)/365)) 

# TEST
test_bene_data['DOD'] = pd.to_datetime(test_bene_data['DOD'])
test_bene_data['Age'] = round(((test_bene_data['DOD'] - test_bene_data['DOB']).dt.days)/365)
test_bene_data['Age'] = test_bene_data['Age'].fillna(round(((test_bene_data['DOD'].max() - test_bene_data['DOB']).dt.days)/365)) 

#making gender binary variable:
# TRAIN 
train_bene_data['Gender'] = train_bene_data['Gender'] - 1

# TEST 
test_bene_data['Gender'] = test_bene_data['Gender'] - 1


# RenalDiseaseIndicator to numeric type:
# TRAIN
rd_indicator = []
for i, row in train_bene_data.iterrows():
    if train_bene_data['RenalDiseaseIndicator'][i] == 'Y':
        rd_indicator.append(1)
    else:
        rd_indicator.append(0)
train_bene_data['RenalDiseaseIndicator'] = rd_indicator

train_bene_data['RenalDiseaseIndicator'] = train_bene_data['RenalDiseaseIndicator'].astype('int64')

# TEST
rd_indicator = []
for i, row in test_bene_data.iterrows():
    if test_bene_data['RenalDiseaseIndicator'][i] == 'Y':
        rd_indicator.append(1)
    else:
        rd_indicator.append(0)
test_bene_data['RenalDiseaseIndicator'] = rd_indicator

test_bene_data['RenalDiseaseIndicator'] = test_bene_data['RenalDiseaseIndicator'].astype('int64')

### Merging dataframes:

In [ ]:
print("These columns are only applicable to inpatient claims:")
for col in train_inp_data.columns:
    if col not in train_outp_data.columns:
        print("-", col)

These columns are only applicable to inpatient claims:
- AdmissionDt
- DischargeDt
- DiagnosisGroupCode
- Duration


In [ ]:
#non numeric code values

codes = ['ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmAdmitDiagnosisCode']

claims_df = [train_outp_data, test_outp_data, train_inp_data, test_inp_data]

for col in codes:
    for df in claims_df:
        this_df = df[pd.notnull(df[col])]
        print(*(this_df[col].astype(str).str[0:1]).unique())

7 6 2 8 V 1 9 4 5 3 0 E
V 9 7 5 8 4 2 1 3 0 6 E
1 6 2 4 0 5 8 9 7 3 V E
5 1 4 8 2 0 7 V 6 9 3 E
V 3 4 7 5 9 2 6 1 E 8 0
V E 2 9 7 3 5 4 6 0 1 8
4 2 3 V 7 E 5 0 9 6 1 8
5 4 7 V 2 3 0 9 E 1 6 8
V 7 2 4 5 3 6 8 E 9 0 1
2 4 3 7 1 V 5 9 6 8 E 0
5 7 3 6 V 2 4 E 9 0 1 8
4 5 6 9 7 2 V 0 3 E 8 1
4 V 7 5 2 3 E 6 9 0 1 8
3 V 2 5 4 7 8 9 1 6 0 E
7 3 5 2 4 V 9 8 0 6 E 1
V 5 4 3 E 7 9 2 6 1 0 8
7 V 2 4 3 9 0 5 E 6 1 8
V 2 6 E 4 7 5 3 1 9 0 8
2 V 4 7 5 6 3 E 1 0 8 9
4 V 9 5 2 7 3 1 8 E 6 0
V 4 3 8 5 7 2 9 0 6 1 E
4 7 5 2 V 3 E 6 9 1 0 8
7 3 4 1 5 2 9 V E 0 6 8
2 5 4 V 3 E 7 9 1 0 6 8
9 V 2 3 4 5 7 E 1 0 6 8
2 V 5 8 3 4 7 1 E 9 6 0
2 5 4 0 V 7 6 E 3 1 9 8
5 4 7 V 3 2 1 9 E 8 0 6
8 7 4 2 9 V E 5 6 1 3 0
5 V 9 4 7 0 2 E 3 6 1 8
1 4 2 5 7 3 V 0 E 6 8 9
5 2 7 4 3 V 1 9 E 0 6 8
7 3 4 2 5 0 V 1 E 6 9 8
7 V 9 2 5 3 4 1 6 E 8 0
5 4 2 3 V 8 1 7 9 6 E 0
4 3 5 9 V 2 7 0 1 6 E 8
V 2 4 1 7 8 5 E 3 9 0 6
4 7 V 1 2 9 3 5 E 8
4 3 V 9 7 5 2 0 1 E 8 6
4 5 7 2 9 3 E 8 V 0 6 1
9 4 6 5 2 8 7 3 1
9 5 7 3 4 6 8
7 3 8 4 9 6 

In [ ]:
#non numeric code values
print(*(train_inp_data['DiagnosisGroupCode'].astype(str).str[0:1]).unique())
print((train_inp_data['DiagnosisGroupCode'][train_inp_data['DiagnosisGroupCode'].str.startswith('O')]).unique())

2 7 8 0 9 5 3 1 4 6 O
['OTH']


In [ ]:
col_list = [col for col in train_outp_data.columns]

train_all_claims = train_outp_data.merge(train_inp_data, how = 'outer', on = col_list)
test_all_claims = train_outp_data.merge(train_inp_data, how = 'outer', on = col_list)

In [ ]:
train_claims_bene_data = train_all_claims.merge(train_bene_data, how = 'outer', on = 'BeneID')
test_claims_bene_data = test_all_claims.merge(train_bene_data, how = 'outer', on = 'BeneID')

In [ ]:
train_all_data = train_claims_bene_data.merge(train_provider_data, how = 'outer', on = 'Provider')
test_all_data = test_claims_bene_data.merge(test_provider_data, how = 'outer', on = 'Provider')

In [ ]:
train_all_in_data = train_all_data[(train_all_data['In_Out'] == 1)]
test_all_in_data = test_all_data[(test_all_data['In_Out'] == 1)]
train_all_out_data = train_all_data[(train_all_data['In_Out'] == 0)]
test_all_out_data = test_all_data[(test_all_data['In_Out'] == 0)]

In [ ]:
train_all_data.to_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/data_out/train_all_df.csv', index=False)
test_all_data.to_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/data_out/test_all_df.csv', index=False)

train_inp_data.to_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/data_out/train_inp_data.csv', index=False)
test_inp_data.to_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/data_out/test_inp_data.csv', index=False)

train_outp_data.to_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/data_out/train_outp_data.csv', index=False)
test_outp_data.to_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/data_out/test_outp_data.csv', index=False)

train_bene_data.to_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/data_out/train_bene_data.csv', index=False)
test_bene_data.to_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/data_out/test_bene_data.csv', index=False)

train_provider_data.to_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/data_out/train_provider_data.csv', index=False)
test_provider_data.to_csv('/Users/Julia/Documents/bootcamp/fraud_capstone/data_out/test_provider_data.csv', index=False)

# Aggregation
### Aggregating information on beneficiaries, according to claims data

In [ ]:
# general information about a provider's patients

train_final_data = train_provider_data.copy()

# mean beneficiary data, duration of stay, and charges
mean_list = ['Age', 'Death', 'Gender', 'Duration', 'RenalDiseaseIndicator', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov', \
        'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease', \
        'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression', \
        'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis', \
        'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke', 'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
       'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt',  'InscClaimAmtReimbursed', 'In_Out']

mean_dict = {0:'Mean_Age', 1:'Mean_Death', 2:'Mean_Gender', 3:'Mean_Duration', 4:'RenalDiseaseIndicator', 5:'NoOfMonths_PartACov', 6:'NoOfMonths_PartBCov', 
        7:'ChronicCond_Alzheimer', 8:'ChronicCond_Heartfailure', 9:'ChronicCond_KidneyDisease', 
        10:'ChronicCond_Cancer', 11:'ChronicCond_ObstrPulmonary', 12:'ChronicCond_Depression', 
        13:'ChronicCond_Diabetes', 14:'ChronicCond_IschemicHeart', 15:'ChronicCond_Osteoporasis', 
        16:'ChronicCond_rheumatoidarthritis', 17:'ChronicCond_stroke', 18:'Mean_IPAnnualReimbursementAmt', 19:'Mean_IPAnnualDeductibleAmt',
       20:'Mean_OPAnnualReimbursementAmt', 21:'Mean_OPAnnualDeductibleAmt',  22:'Mean_InscClaimAmtReimbursed', 23:'Mean_In_Out'}

mean_tuples = list(zip(mean_list, range(0, 24)))

for col_, num in mean_tuples:
    grouped = (train_all_data.groupby(['Provider'])[col_].mean()).reset_index()
    grouped = grouped.rename(index=str, columns={col_:mean_dict[num]})
    train_final_data = pd.merge(grouped, train_final_data, on='Provider')

# race columns
max_one_race = train_all_data.groupby(['Provider', 'Race'])['Race'].count().groupby('Provider').max()
all_people = train_all_data.groupby(['Provider', 'Race'])['Race'].count().groupby('Provider').sum()
race_ratio = max_one_race / all_people
train_final_data = pd.merge(race_ratio.to_frame('High_Race_Perc'), train_final_data, left_index=True, right_on='Provider')

def mode_col(col, t_data):
    name_ = t_data.groupby(['Provider', col])[col].count().groupby('Provider').idxmax()
    list_ = [x[1] for x in name_]
    train_final_data[col] = list_
    
mode_col('Race', train_all_data)
mode_col('State', train_all_data)
mode_col('County', train_all_data)


In [ ]:
# How much diversity is there with the type of care provided?
# We count the unique values in each category per provider and divide by number of claims

count_dict = {2:'U_AttendingPhysician', 3:'U_OperatingPhysician', 4:'U_OtherPhysician', 
             5:'U_ClmDiagnosisCode_1', 6:'U_ClmDiagnosisCode_2',
             7:'U_ClmDiagnosisCode_3', 8:'U_ClmDiagnosisCode_4', 9:'U_ClmDiagnosisCode_5',
             10:'U_ClmDiagnosisCode_6', 11:'U_ClmDiagnosisCode_7', 12:'U_ClmDiagnosisCode_8',
             13:'U_ClmDiagnosisCode_9', 14:'U_ClmDiagnosisCode_10', 15:'U_ClmProcedureCode_1',
             16:'U_ClmProcedureCode_2', 17:'U_ClmProcedureCode_3', 18:'U_ClmAdmitDiagnosisCode', 
             19:'U_DiagnosisGroupCode', 20: 'Mean_Reimbursement'}

count_cols = ['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician', 
             'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
             'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
             'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
             'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
             'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmAdmitDiagnosisCode', 
             'DiagnosisGroupCode', 'InscClaimAmtReimbursed']

count_tuples = list(zip(count_cols, range(2, 21)))

for col_, num in count_tuples:
    grouped = train_all_data[['Provider', col_]].groupby(['Provider']).count().reset_index()
    div = train_all_data.groupby(['Provider'])['ClaimID'].count().reset_index()
    grouped[col_] = grouped[col_]/div['ClaimID']
    grouped = grouped.rename(index=str, columns={col_:count_dict[num]})
    train_final_data = pd.merge(grouped, train_final_data, on="Provider" )


In [ ]:
# How many claims is the provider processing?
grouped = train_all_data.groupby(['Provider', 'ClaimID'])['ClaimID'].count().groupby('Provider').sum().reset_index()
grouped = grouped.rename(index=str, columns={'ClaimID':'Sum_ClaimID'})
train_final_data = pd.merge(grouped, train_final_data, on="Provider")


In [ ]:
# We will use ClmDiagnosisCode_1 to classify outpatient claims.
# Will will use DiagnosisGroupCode to classify inpatient claims
# Will will only use values in the keep_list, as these values have over 20 observations.

series = train_all_out_data[['ClmDiagnosisCode_1', 'ClaimID']].groupby('ClmDiagnosisCode_1').count().reset_index()
keep_list_out = list((series['ClmDiagnosisCode_1'][series['ClaimID'] >= 30]))

series = train_all_in_data[['DiagnosisGroupCode', 'ClaimID']].groupby('DiagnosisGroupCode').count().reset_index()
keep_list_in = list((series['DiagnosisGroupCode'][series['ClaimID'] >= 30]))


In [ ]:
# Are physicians getting higher reimbursements than expected? 
# What's the maximum average reimbursement a physician makes per Diagnosis Group Code?

#inpatient
train_all_in_data_agg = train_all_in_data[(train_all_in_data['DiagnosisGroupCode'].isin(keep_list_in))]
avgs = train_all_in_data_agg.groupby(['Provider', 'DiagnosisGroupCode', 'AttendingPhysician'])['InscClaimAmtReimbursed'].mean().reset_index().drop('AttendingPhysician', axis=1)
in_maxs = avgs.groupby(['Provider', 'DiagnosisGroupCode'])['InscClaimAmtReimbursed'].max().reset_index()
in_maxs = in_maxs.pivot(index='Provider', columns='DiagnosisGroupCode', values='InscClaimAmtReimbursed')
in_maxs = in_maxs.add_prefix('in_')
train_final_data = pd.merge(in_maxs, train_final_data, how='right', on = 'Provider')

#outpatient
train_all_out_data_agg = train_all_out_data[(train_all_out_data['ClmDiagnosisCode_1'].isin(keep_list_out))]                                         
avgs = train_all_out_data_agg.groupby(['Provider', 'ClmDiagnosisCode_1', 'AttendingPhysician'])['InscClaimAmtReimbursed'].mean().reset_index().drop('AttendingPhysician', axis=1)
out_maxs = avgs.groupby(['Provider', 'ClmDiagnosisCode_1'])['InscClaimAmtReimbursed'].max().reset_index()
out_maxs = out_maxs.pivot(index='Provider', columns='ClmDiagnosisCode_1', values='InscClaimAmtReimbursed')
out_maxs = out_maxs.add_prefix('out_')
train_final_data = pd.merge(out_maxs, train_final_data, how='right', on = 'Provider')

In [ ]:
# Are physicians doing duplicate claims per patient?
# Look for the maximum number of repeated pairs per provider

counts = train_all_data.groupby(['Provider', 'AttendingPhysician', 'BeneID'])['ClaimID'].count()
counts_maxs = counts.groupby(['Provider']).max().reset_index()
counts_maxs.columns = ['Provider', 'Phys_Bene_Repeats']
train_final_data = pd.merge(counts_maxs, train_final_data, on = 'Provider')


In [ ]:
train_final_data.head()

In [ ]:
train_final_data.shape